In [1]:
import pickle
import numpy as np
import pandas as pd
import requests
from tqdm.auto import tqdm

from rectools.models import PopularModel
from rectools.dataset import Interactions, Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools import Columns
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics
from service.userknn import UserKnn


In [ ]:
import zipfile
import os

url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...', 
                        total=total_size_in_bytes, 
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

zip_file_path = './kion_train.zip'
output_directory = '../artifacts'
os.makedirs(output_directory, exist_ok=True)
desired_files = ['first_reco_result.csv', 'interactions.csv', 'items.csv']

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        if file_info.filename in desired_files:
            destination_path = os.path.join(output_directory, file_info.filename)
            zip_ref.extract(file_info, output_directory)


In [ ]:
interactions_df = pd.read_csv('../artifacts/interactions.csv')
users = pd.read_csv('../artifacts/users.csv')
items = pd.read_csv('../artifacts/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True) 

# Мы хотим учитывать юзеров с 1 или двумя просмотрами, так как этого недостаточно чтобы говорить о пользователях хоть что-нибудь
user_ids_all = interactions_df.groupby('user_id')['item_id'].nunique().reset_index(name='unique_items_count')
hot_users = user_ids_all[user_ids_all['unique_items_count'] > 20]['user_id']
interactions_df_hot_users = interactions_df[interactions_df['user_id'].isin(hot_users)]

interactions = Interactions(interactions_df_hot_users)   
interactions.df

In [ ]:
N_SPLITS = 3
TEST_SIZE = '14D'

cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=False,
)
cv.get_test_fold_borders(interactions)

In [ ]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10)
}

# few simple models to compare
models = {
    'cosine_userknn': CosineRecommender(),
    'tfidf_userknn': TFIDFRecommender(), 
}
TFIDFRecommender.recommend

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

models_wrps = {}
for model_name, model in models.items():
    models_wrps[model_name] = UserKnn(model=model, N_users=50)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    print(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = interactions.df[Columns.Item].unique()
    
    for model_name, userknn_model in models_wrps.items():
        userknn_model.fit(interactions.df)
    
        recos = userknn_model.predict(df_test)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=interactions.df,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)

In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics.groupby('model').mean()[metrics.keys()]

In [ ]:
for model_name, model_wrp in models_wrps.items():
    with open(f"../artifacts/task3_cropped20_experiment_{model_name}.pkl", "wb") as file:
        pickle.dump(model_wrp, file)

In [ ]:
with open("../artifacts/task3_cropped20_experiment_tfidf_userknn.pkl", "rb") as file:
    model_1 = pickle.load(file)


user_id_kostyl = pd.DataFrame({'user_id': [774973]})
recos = model_1.predict(user_id_kostyl)

    
metric_values = calc_metrics(
    metrics,
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog,
)

In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics.groupby('model').mean()[metrics.keys()]

In [ ]:
hot_users